In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
# Evaluate metrics on the test set
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef

# **Load data**

In [6]:
import numpy as np
train_actives = np.load('Path to npy file')
test_actives = np.load('Path to npy file')
train_decoys= np.load('Path to npy file')
test_decoys = np.load('Path to npy file')
test_actives_hard_test_1 = np.load('Path to npy file')
test_actives_hard_test_2 = np.load('Path to npy file')
print(train_actives.shape)
print(test_actives.shape)
print(test_actives_hard_test_1.shape)
print(test_actives_hard_test_2.shape)
print(train_decoys.shape)
print(test_decoys.shape)

(651, 4, 4, 4, 1539)
(85, 4, 4, 4, 1539)
(53, 4, 4, 4, 1539)
(29, 4, 4, 4, 1539)
(32499, 4, 4, 4, 1536)
(4250, 4, 4, 4, 1536)


In [2]:
import numpy as np
train_actives = np.load('/home/juni/working/mettl3/voxels_data/grid_train_actives_splif.npy')
test_actives = np.load('/home/juni/working/mettl3/voxels_data/grid_test_actives_splif.npy')
train_decoys= np.load('/home/juni/working/mettl3/voxels_data/grid_train_decoys_splif_full.npy')
test_decoys = np.load('/home/juni/working/mettl3/voxels_data/grid_test_decoys_splif_full.npy')
test_actives_hard_test_1 = np.load('/home/juni/working/mettl3/voxels_data/grid_test_actives_splif_filtered_80.npy')
test_actives_hard_test_2 = np.load('/home/juni/working/mettl3/voxels_data/grid_test_actives_splif_filtered_75.npy')
print(train_actives.shape)
print(test_actives.shape)
print(test_actives_hard_test_1.shape)
print(test_actives_hard_test_2.shape)
print(train_decoys.shape)
print(test_decoys.shape)

(651, 4, 4, 4, 1539)
(85, 4, 4, 4, 1539)
(53, 4, 4, 4, 1539)
(29, 4, 4, 4, 1539)
(32499, 4, 4, 4, 1536)
(4250, 4, 4, 4, 1536)


In [3]:
train_actives = train_actives[:, :, :, :, :-3]
test_actives = test_actives[:, :, :, :, :-3]
test_actives_hard_test_1 = test_actives_hard_test_1[:, :, :, :, :-3]
test_actives_hard_test_2 = test_actives_hard_test_2[:, :, :, :, :-3]

In [ ]:



from torch.utils.data import TensorDataset, DataLoader



# **Reshaping for CNN Input:**
To feed this data into a 3D CNN in PyTorch, you typically need to structure it in the format (N, C, D, H, W), where:

N is the batch size (number of samples).\
C is the number of channels/features (in this case, 1539).\
D, H, W are the depth, height, and width of the 3D grid (in this case, 4, 4, 4).

In [4]:
import torch

# Reshape the data for PyTorch CNN input
train_actives_tensor = torch.tensor(train_actives, dtype=torch.float32)
test_actives_tensor = torch.tensor(test_actives, dtype=torch.float32)
test_actives_hard_test_1_tensor = torch.tensor(test_actives_hard_test_1, dtype=torch.float32)
test_actives_hard_test_2_tensor = torch.tensor(test_actives_hard_test_2, dtype=torch.float32)
train_decoys_tensor = torch.tensor(train_decoys, dtype=torch.float32)
test_decoys_tensor = torch.tensor(test_decoys, dtype=torch.float32)


train_actives_tensor = train_actives_tensor.permute(0, 4, 1, 2, 3)  # Shape will be (651, 1539, 4, 4, 4)
test_actives_tensor = test_actives_tensor.permute(0, 4, 1, 2, 3)  # Shape will be (651, 1539, 4, 4, 4)
test_actives_hard_test_1_tensor = test_actives_hard_test_1_tensor.permute(0, 4, 1, 2, 3)  # Shape will be (651, 1539, 4, 4, 4)
test_actives_hard_test_2_tensor = test_actives_hard_test_2_tensor.permute(0, 4, 1, 2, 3)
train_decoys_tensor = train_decoys_tensor.permute(0, 4, 1, 2, 3)  # Shape will be (651, 1539, 4, 4, 4)
test_decoys_tensor = test_decoys_tensor.permute(0, 4, 1, 2, 3)  # Shape will be (651, 1539, 4, 4, 4)

print(train_actives_tensor.shape)
print(test_actives_tensor.shape)
print(test_actives_hard_test_1_tensor.shape)
print(test_actives_hard_test_2_tensor.shape)
print(train_decoys_tensor.shape)
print(test_decoys_tensor.shape)

torch.Size([651, 1536, 4, 4, 4])
torch.Size([85, 1536, 4, 4, 4])
torch.Size([53, 1536, 4, 4, 4])
torch.Size([29, 1536, 4, 4, 4])
torch.Size([32499, 1536, 4, 4, 4])
torch.Size([4250, 1536, 4, 4, 4])


In [6]:
# Concatenate training data
train_data = torch.cat((train_actives_tensor, train_decoys_tensor), dim=0)  # Shape: (1302, 4, 4, 4, 1539)

# Concatenate test data
test_data = torch.cat((test_actives_tensor, test_decoys_tensor), dim=0)  # Shape: (170, 4, 4, 4, 1539)
hard_test_data_1 = torch.cat((test_actives_hard_test_1_tensor, test_decoys_tensor), dim=0)  # Shape: (170, 4, 4, 4, 1539)
hard_test_data_2 = torch.cat((test_actives_hard_test_2_tensor, test_decoys_tensor), dim=0)  # Shape: (170, 4, 4, 4, 1539)
print(train_data.shape)
print(test_data.shape)
print(hard_test_data_1.shape)
print(hard_test_data_2.shape)

torch.Size([33150, 1536, 4, 4, 4])
torch.Size([4335, 1536, 4, 4, 4])
torch.Size([4303, 1536, 4, 4, 4])
torch.Size([4279, 1536, 4, 4, 4])


In [7]:
# Create labels for training data
train_labels = torch.cat((torch.ones(651), torch.zeros(32499)), dim=0)  # Shape: (1302,)

# Create labels for test data
test_labels = torch.cat((torch.ones(85), torch.zeros(4250)), dim=0)  # Shape: (170,)
hard_test_1_labels = torch.cat((torch.ones(53), torch.zeros(4250)), dim=0)  # Shape: (170,)
hard_test_2_labels = torch.cat((torch.ones(29), torch.zeros(4250)), dim=0)  # Shape: (170,)


print(train_labels.shape)
print(test_labels.shape)
print(hard_test_1_labels.shape)
print(hard_test_2_labels.shape)

torch.Size([33150])
torch.Size([4335])
torch.Size([4303])
torch.Size([4279])


In [9]:
# Shuffle training data and labels together
train_indices = torch.randperm(train_data.size(0))
train_data = train_data[train_indices]
train_labels = train_labels[train_indices]

# Shuffle test data and labels together
test_indices = torch.randperm(test_data.size(0))
test_data = test_data[test_indices]
test_labels = test_labels[test_indices]

# Shuffle hard test data 1 and labels together
test_indices_filtered = torch.randperm(hard_test_data_1.size(0))
test_data_filtered = hard_test_data_1[test_indices_filtered]
test_labels_filtered = hard_test_1_labels[test_indices_filtered]

# Shuffle hard test data 2 and labels together
test_indices_filtered_1 = torch.randperm(hard_test_data_2.size(0))
test_data_filtered_1 = hard_test_data_2[test_indices_filtered_1]
test_labels_filtered_1 = hard_test_2_labels[test_indices_filtered_1]

In [10]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDataset for training and testing
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)
test_dataset_filtered = TensorDataset(test_data_filtered, test_labels_filtered)
test_dataset_filtered_1 = TensorDataset(test_data_filtered_1, test_labels_filtered_1)


# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
test_loader_filtered = DataLoader(test_dataset_filtered, batch_size=32, shuffle=False)
test_loader_filtered_1 = DataLoader(test_dataset_filtered_1, batch_size=32, shuffle=False)

In [11]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [12]:
import numpy as np
def set_Seed(seed=41):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_Seed(42)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class MultiheadAttention3D(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiheadAttention3D, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # Reshape (batch_size, channels, D, H, W) -> (batch_size, channels, D*H*W)
        batch_size, channels, D, H, W = x.shape
        x = x.view(batch_size, channels, -1).permute(2, 0, 1)  # Shape: (D*H*W, batch_size, channels)

        # Apply Multihead Attention
        attn_output, _ = self.multihead_attn(x, x, x)
        attn_output = self.norm(attn_output)

        # Reshape back to (batch_size, channels, D, H, W)
        attn_output = attn_output.permute(1, 2, 0).view(batch_size, channels, D, H, W)
        return attn_output

class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        # Input shape: (batch_size, 1539, 4, 4, 4)
        self.conv1 = nn.Conv3d(1536, 32, kernel_size=3, stride=1, padding=1)  # Output: (batch_size, 512, 4, 4, 4)
        self.bn1 = nn.BatchNorm3d(32)

        #self.attn1 = MultiheadAttention3D(embed_dim=32, num_heads=1)  # Apply Multihead Attention
        self.attn_layers = nn.ModuleList([MultiheadAttention3D(embed_dim=32, num_heads=8) for _ in range(6)])  # Apply Multihead Attention with 6 layers
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: (batch_size, 256, 4, 4, 4)
        self.bn2 = nn.BatchNorm3d(64)

        self.attn2 = MultiheadAttention3D(embed_dim=64, num_heads=8)  # Apply Multihead Attention

        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)  # Output: (batch_size, 128, 4, 4, 4)
        self.bn3 = nn.BatchNorm3d(128)

        self.attn3 = MultiheadAttention3D(embed_dim=128, num_heads=8)  # Apply Multihead Attention

        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)  # Output: (batch_size, 128, 2, 2, 2)
        self.dropout_conv = nn.Dropout3d(p=0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 2 * 2 * 2, 256)
        self.dropout_fc1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 128)
        self.dropout_fc2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(128, 1)  # Binary classification (e.g., active/inactive)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # Conv1 + BatchNorm + ReLU
        #Apply 6 layers of Multihead Attention sequentially
        for attn_layer in self.attn_layers:
             x = attn_layer(x)
        #x = self.attn1(x)  # Multihead Attention after conv1

        x = F.relu(self.bn2(self.conv2(x)))  # Conv2 + BatchNorm + ReLU
        #x = self.attn2(x)  # Multihead Attention after conv2

        x = F.relu(self.bn3(self.conv3(x)))  # Conv3 + BatchNorm + ReLU
        #x = self.attn3(x)  # Multihead Attention after conv3

        x = self.dropout_conv(x)  # Dropout after convolutions
        x = self.pool(x)  # Pooling layer

        x = x.view(x.size(0), -1)  # Flatten the output
        x = F.relu(self.fc1(x))  # Fully connected layer 1
        x = self.dropout_fc1(x)  # Dropout after fc1
        x = F.relu(self.fc2(x))  # Fully connected layer 2
        x = self.dropout_fc2(x)  # Dropout after fc2
        x = torch.sigmoid(self.fc3(x))  # Sigmoid for binary classification
        #x = self.fc3(x)  # No activation function for regression
        return x
# Example usage:
model = CNN3D()
print(model)

CNN3D(
  (conv1): Conv3d(1536, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (attn_layers): ModuleList(
    (0-5): 6 x MultiheadAttention3D(
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
      )
      (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (attn2): MultiheadAttention3D(
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
    )
    (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (conv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn3): BatchNorm3

In [16]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef
from torch.utils.data import DataLoader, Subset
import numpy as np

# Assuming `train_loader` is the original full dataset.
# You can split it directly into training and validation datasets as below

# Split the dataset into training and validation (80% for training and 20% for validation)
train_size = int(0.8 * len(train_loader.dataset))  # 80% for training
val_size = len(train_loader.dataset) - train_size  # 20% for validation

# Use random_split to split the data
train_dataset, val_dataset = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

# Create DataLoader for training and validation
train_loader_split = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader_split = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the model and optimizer
model = CNN3D().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
criterion = nn.BCELoss()

num_epochs = 100
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Initialize dictionaries to store metrics for training and validation
train_metrics = {
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1_score": [],
    "roc_auc": [],
    "pr_auc": [],
    "mcc": []
}

val_metrics = {
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1_score": [],
    "roc_auc": [],
    "pr_auc": [],
    "mcc": []
}

# Training and validation loops
for epoch in range(num_epochs):
    # Training Loop
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    train_predictions = []
    train_labels = []

    for inputs, targets in train_loader_split:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = (outputs.squeeze() >= 0.5).float()
        train_predictions.extend(predicted.cpu().numpy())
        train_labels.extend(targets.cpu().numpy())
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    epoch_loss = running_loss / len(train_loader_split)
    epoch_accuracy = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    # Validation Loop
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for inputs, targets in val_loader_split:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            val_loss = criterion(outputs.squeeze(), targets)
            val_running_loss += val_loss.item()
            predicted = (outputs.squeeze() >= 0.5).float()
            val_predictions.extend(predicted.cpu().numpy())
            val_labels.extend(targets.cpu().numpy())
            val_correct += (predicted == targets).sum().item()
            val_total += targets.size(0)

    val_loss_epoch = val_running_loss / len(val_loader_split)
    val_accuracy_epoch = val_correct / val_total
    val_losses.append(val_loss_epoch)
    val_accuracies.append(val_accuracy_epoch)

    # Compute validation metrics
    val_precision = precision_score(val_labels, val_predictions, zero_division=0)
    val_recall = recall_score(val_labels, val_predictions, zero_division=0)
    val_f1 = f1_score(val_labels, val_predictions, zero_division=0)
    val_roc_auc = roc_auc_score(val_labels, val_predictions)
    val_pr_auc = average_precision_score(val_labels, val_predictions)
    val_mcc = matthews_corrcoef(val_labels, val_predictions)

    # Store validation metrics for this epoch
    val_metrics["accuracy"].append(val_accuracy_epoch)
    val_metrics["precision"].append(val_precision)
    val_metrics["recall"].append(val_recall)
    val_metrics["f1_score"].append(val_f1)
    val_metrics["roc_auc"].append(val_roc_auc)
    val_metrics["pr_auc"].append(val_pr_auc)
    val_metrics["mcc"].append(val_mcc)

    # Compute training metrics
    train_precision = precision_score(train_labels, train_predictions, zero_division=0)
    train_recall = recall_score(train_labels, train_predictions, zero_division=0)
    train_f1 = f1_score(train_labels, train_predictions, zero_division=0)
    train_roc_auc = roc_auc_score(train_labels, train_predictions)
    train_pr_auc = average_precision_score(train_labels, train_predictions)
    train_mcc = matthews_corrcoef(train_labels, train_predictions)

    # Store training metrics for this epoch
    train_metrics["accuracy"].append(epoch_accuracy)
    train_metrics["precision"].append(train_precision)
    train_metrics["recall"].append(train_recall)
    train_metrics["f1_score"].append(train_f1)
    train_metrics["roc_auc"].append(train_roc_auc)
    train_metrics["pr_auc"].append(train_pr_auc)
    train_metrics["mcc"].append(train_mcc)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_accuracy:.4f}, Val Loss: {val_loss_epoch:.4f}, Val Acc: {val_accuracy_epoch:.4f}, Val F1: {val_f1:.4f}, Val ROC-AUC: {val_roc_auc:.4f}')

# After training, you can directly plot or store the results for each epoch, such as accuracy, loss, etc.


Epoch 1/100, Train Loss: 0.0215, Train Acc: 0.9937, Val Loss: 0.0027, Val Acc: 0.9991, Val F1: 0.9790, Val ROC-AUC: 0.9860
Epoch 2/100, Train Loss: 0.0057, Train Acc: 0.9987, Val Loss: 0.0045, Val Acc: 0.9989, Val F1: 0.9753, Val ROC-AUC: 0.9791
Epoch 3/100, Train Loss: 0.0006, Train Acc: 0.9998, Val Loss: 0.0056, Val Acc: 0.9994, Val F1: 0.9861, Val ROC-AUC: 0.9929
Epoch 4/100, Train Loss: 0.0001, Train Acc: 1.0000, Val Loss: 0.0062, Val Acc: 0.9992, Val F1: 0.9826, Val ROC-AUC: 0.9894
Epoch 5/100, Train Loss: 0.0064, Train Acc: 0.9987, Val Loss: 0.0054, Val Acc: 0.9985, Val F1: 0.9648, Val ROC-AUC: 0.9755
Epoch 6/100, Train Loss: 0.0016, Train Acc: 0.9996, Val Loss: 0.0079, Val Acc: 0.9989, Val F1: 0.9759, Val ROC-AUC: 0.9927
Epoch 7/100, Train Loss: 0.0002, Train Acc: 1.0000, Val Loss: 0.0155, Val Acc: 0.9991, Val F1: 0.9792, Val ROC-AUC: 0.9894
Epoch 8/100, Train Loss: 0.0000, Train Acc: 1.0000, Val Loss: 0.0166, Val Acc: 0.9991, Val F1: 0.9793, Val ROC-AUC: 0.9927
Epoch 9/100, Tra

In [17]:
import os

# Define the directory where you want to save the model
save_dir = './models'  # You can change this to your desired directory path

# Ensure the directory exists
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define the path where the model will be saved
save_path = os.path.join(save_dir, 'model_checkpoint_predict_100epochs.pth')

# Save the model state_dict and optimizer state_dict (if needed)
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': running_loss,
}, save_path)

print(f"Model saved to {save_path}")

Model saved to ./models/model_checkpoint_predict_100epochs.pth


In [22]:
import pandas as pd
print("test set")# Final model evaluation on a completely different test set
# Final model evaluation on a completely different test set
model.eval()
test_correct = 0
test_total = 0
test_predictions = []
test_labels = []
test_output = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        test_output.extend(outputs.cpu().numpy().astype(float))
        predicted = (outputs.squeeze() >= 0.5).float()
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(targets.cpu().numpy())
        test_correct += (predicted == targets).sum().item()
        test_total += targets.size(0)

test_accuracy = test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.4f}')

# Evaluate metrics on the test set
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef

precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)
roc_auc = roc_auc_score(test_labels, test_predictions)
pr_auc = average_precision_score(test_labels, test_predictions)
mcc = matthews_corrcoef(test_labels, test_predictions)

print(f'PR-AUC: {pr_auc:.4f}')
print(f'ROC-AUC: {roc_auc:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'MCC: {mcc:.4f}')

test_results_drA = pd.DataFrame({"Predicted_score": test_output})
test_results_drA['Predicted_Activity'] = test_predictions
test_results_drA['Observed_Activity'] = test_labels
test_results_drA.to_csv("./final_model_results/test_VS_run_5.csv", index=False)


print("Hard test 1")
# Final model evaluation on a completely different test set
model.eval()
test_correct = 0
test_total = 0
test_predictions = []
test_labels = []
test_output = []

with torch.no_grad():
    for inputs, targets in test_loader_filtered:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        test_output.extend(outputs.cpu().numpy())
        predicted = (outputs.squeeze() >= 0.5).float()
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(targets.cpu().numpy())

# Evaluate metrics on the test set
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef,precision_recall_curve

precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)
roc_auc = roc_auc_score(test_labels, test_predictions)
pr_auc = average_precision_score(test_labels, test_predictions)
mcc = matthews_corrcoef(test_labels, test_predictions)

print(f'PR-AUC: {pr_auc:.4f}')
print(f'ROC-AUC: {roc_auc:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'MCC: {mcc:.4f}')

hard_test_1_results_drA = pd.DataFrame({"Predicted_score": test_output})
hard_test_1_results_drA['Predicted_Activity'] = test_predictions
hard_test_1_results_drA['Observed_Activity'] = test_labels
hard_test_1_results_drA.to_csv("./final_model_results/hard_test_1_VS_run_5.csv", index=False)


print("Hard test 2")
# Final model evaluation on a completely different test set
model.eval()
test_correct = 0
test_total = 0
test_predictions = []
test_labels = []
test_output = []

with torch.no_grad():
    for inputs, targets in test_loader_filtered_1:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        test_output.extend(outputs.cpu().numpy().astype(float))
        #test_output.extend(outputs.cpu().numpy())
        predicted = (outputs.squeeze() >= 0.5).float()
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(targets.cpu().numpy())

# Evaluate metrics on the test set
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef

precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)
roc_auc = roc_auc_score(test_labels, test_predictions)
pr_auc = average_precision_score(test_labels, test_predictions)
mcc = matthews_corrcoef(test_labels, test_predictions)

print(f'PR-AUC: {pr_auc:.4f}')
print(f'ROC-AUC: {roc_auc:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'MCC: {mcc:.4f}')

hard_test_2_results_drA = pd.DataFrame({"Predicted_score": test_output})
hard_test_2_results_drA['Predicted_Activity'] = test_predictions
hard_test_2_results_drA['Observed_Activity'] = test_labels
hard_test_2_results_drA.to_csv("./final_model_results/hard_test_2_VS_run_5.csv", index=False)

test set
Test Accuracy: 0.9995
PR-AUC: 0.9768
ROC-AUC: 0.9940
Precision: 0.9882
Recall: 0.9882
F1 Score: 0.9882
MCC: 0.9880
Hard test 1
PR-AUC: 0.9629
ROC-AUC: 0.9904
Precision: 0.9811
Recall: 0.9811
F1 Score: 0.9811
MCC: 0.9809
Hard test 2
PR-AUC: 0.9325
ROC-AUC: 0.9826
Precision: 0.9655
Recall: 0.9655
F1 Score: 0.9655
MCC: 0.9653


In [24]:
hard_test_1_results_drA

,Predicted_score,Predicted_Activity,Observed_Activity
0,[1.8371831e-07],0.0,0.0
1,[3.4420188e-07],0.0,0.0
2,[4.286056e-07],0.0,0.0
3,[2.1032638e-07],0.0,0.0
4,[1.5208157e-07],0.0,0.0
...,...,...,...
4298,[2.1690974e-07],0.0,0.0
4299,[6.223316e-07],0.0,0.0
4300,[1.9869651e-07],0.0,0.0
4301,[1.4444923e-07],0.0,0.0


In [25]:
import json

# Data structure to save
results = {
    "fold_train_losses": train_losses,
    "fold_val_losses": val_losses,
    "fold_train_accuracies": train_accuracies,
    "fold_val_accuracies": val_accuracies
}

# Save data to a JSON file
with open("./final_model_results/fold_results_run_5.json", "w") as f:
    json.dump(results, f)

print("Results saved to fold_results.json")

# To load the data later
with open("./final_model_results/fold_results_run_5.json", "r") as f:
    loaded_results = json.load(f)

# Accessing loaded data
loaded_fold_train_losses = loaded_results["fold_train_losses"]
loaded_fold_val_losses = loaded_results["fold_val_losses"]
loaded_fold_train_accuracies = loaded_results["fold_train_accuracies"]
loaded_fold_val_accuracies = loaded_results["fold_val_accuracies"]

# # Verify loaded data
# print("Loaded Training Losses:", loaded_fold_train_losses)


Results saved to fold_results.json


In [26]:
import pandas as pd
print('Training')
train_metrics_pd = pd.DataFrame(train_metrics)
train_metrics_pd.to_csv("./final_model_results/train_metrics_run_5.csv")
train_metrics_mean = train_metrics_pd.applymap(lambda x: sum(x) / len(x) if isinstance(x,list) else x)
overallmean = train_metrics_mean.mean()
print(f'PR-AUC: {overallmean[5]:.4f}')
print(f'ROC-AUC: {overallmean[4]:.4f}')
print(f'Precision: {overallmean[1]:.4f}')
print(f'Recall: {overallmean[2]:.4f}')
print(f'F1 Score: {overallmean[3]:.4f}')
print(f'MCC: {overallmean[6]:.4f}')

print('Validation')
val_metrics_pd = pd.DataFrame(val_metrics)
val_metrics_pd.to_csv("./final_model_results/val_metrics_run_5.csv")
val_metrics_mean = val_metrics_pd.applymap(lambda x: sum(x) / len(x) if isinstance(x,list) else x)
overallmean = val_metrics_mean.mean()
print(f'PR-AUC: {overallmean[5]:.4f}')
print(f'ROC-AUC: {overallmean[4]:.4f}')
print(f'Precision: {overallmean[1]:.4f}')
print(f'Recall: {overallmean[2]:.4f}')
print(f'F1 Score: {overallmean[3]:.4f}')
print(f'MCC: {overallmean[6]:.4f}')

Training
PR-AUC: 0.9877
ROC-AUC: 0.9959
Precision: 0.9954
Recall: 0.9918
F1 Score: 0.9935
MCC: 0.9934
Validation
PR-AUC: 0.9206
ROC-AUC: 0.9678
Precision: 0.9824
Recall: 0.9359
F1 Score: 0.9584
MCC: 0.9580
